In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
from utils import test_trainer
#import importlib

print('This is a debug file')

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, Reshape, Conv2D, MaxPool2D, Flatten
from tensorflow.keras import Sequential

model = Sequential()
# Input Layer
model.add(Input(shape=(784,)))
# Reshape Layer
model.add(Reshape((28, 28, 1)))
# Conv Layer
model.add(Conv2D(24, 5, padding='same', activation='relu', input_shape=(28, 28, 1)))
# MaxPool Layer
model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))
# Flatten Layer
model.add(Flatten())
# Dense Layer
model.add(Dense(256, 'relu'))
# Output Layer
model.add(Dense(10, 'softmax'))

#opt = tf.keras.optimizers.SGD(learning_rate=lr)
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
model.compile(opt, loss_fn, metrics=['accuracy'])

In [ ]:
import numpy as np
# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

In [ ]:
model = tf.keras.Sequential(
    [
        tf.keras.Input(shape=input_shape),
        tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(num_classes, activation="softmax"),
    ]
)
model.summary()

In [ ]:
batch_size = 128
epochs = 15

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

In [ ]:
import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras import Sequential

model = Sequential()
# Input Layer
model.add(Input(shape=(784,)))
# Output Layer
model.add(Dense(10, 'softmax', kernel_regularizer=tf.keras.regularizers.l2(0.001)))
    
opt = tf.keras.optimizers.SGD(learning_rate=0.003)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
model.compile(opt, loss_fn, metrics=['accuracy'])

In [ ]:
x = model.get_weights()
print(len(x[0]))